# Subregions and Edges

This code was used to create the masks used in zonaton for the inland continental subregions of the 2023 Blueprint

In [1]:
import os, fnmatch
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
# this is my scratch workspace
OutWorkspace = r"E:\WORKING\BaseBlueprint\2022\4_SubregionsAndEdges\SubregionsAndEdgesForWorkshops.gdb"
# this is where the all the zonaiton masks will go so I can easily zip up to put on teams
OutWorkspace2 = r"E:\WORKING\BaseBlueprint\2022\4_SubregionsAndEdges\SubregionsAndEdgesForWorkshops"
# I'm putting roads in the root directory to make it easier to zip it seperate from the masks
OutWorkspace3 = r"E:\WORKING\BaseBlueprint\2022\4_SubregionsAndEdges"

In [4]:
# define rasters used for cell size, extent, and snapping
SEraster= r"E:\WORKING\BaseBlueprint\2022\2_Subregions\BaseBlueprintExtent2022.tif"

In [5]:
# define inputs
nlcd= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
subr= r"E:\WORKING\BaseBlueprint\2022\2_Subregions\BaseBlueprintSubRgn.shp"
resNull= r"E:\WORKING\BaseBlueprint\2022\3_Reservoirs\BaseBP_2022_ResNull.tif"
roads= r"F:\GIS_DATA\Transporation\OpenStreetMap20220223\MainRoadsAE_20220223.shp"

### Start Analysis

In [6]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [7]:
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\4_SubregionsAndEdges\SubregionsAndEdgesForWorkshops.gdb


### Wrangle subregions
Now that our extent is so large, we can no longer just input the full extent into our zonation subregion runs. We have to generate masks that have the extent for each subregion, and then we need to clip all the indicators for that zonation run to that smaller mask area as well. Blah.

In [8]:
# it would be nice if I could just overwrite in the steps below, because Rua needs both the subregion I and the subregion II 
# for testing, but some subregion I areas are not subdivided again into a subregion II (not east to do with ecoregions and 
#HUCS for those). Here I am checking to see if the environments should allow overwriting
print (arcpy.env.overwriteOutput)

True


In [9]:
# make individual feature classes for each level 1 subregion
arcpy.analysis.SplitByAttributes(subr, OutWorkspace, "SubRgn_I")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_SubregionsAndEdges\\SubregionsAndEdgesForWorkshops.gdb'>

In [10]:
# make individual feature classes for each level 2 subregion
# dang, looking at results, it did not overwrite, instead it made copies of the Subregion I and II that are identical and just put 
# a 0 at the end of the name. Doh
arcpy.analysis.SplitByAttributes(subr, OutWorkspace, "SubRgn_II")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_SubregionsAndEdges\\SubregionsAndEdgesForWorkshops.gdb'>

In [11]:
# make a list of all feature classes
FCList = arcpy.ListFeatureClasses() 

In [12]:
#Print the list made in the above step to see what I found
print(FCList)

['Appalachians', 'Atlantic_Coastal_Plain', 'Chihuahuan_Deserts', 'East_Gulf_Coastal_Plain', 'Edwards_Plateau', 'Florida_Peninsula', 'Gulf_Coastal_Prairies', 'High_Plains_and_Tablelands', 'Interior_Plateau', 'Marine_Gulf_Stream', 'Marine_Shelf_and_Extension', 'Mississippi_Alluvial_Valley', 'North_Missouri', 'Ouachita', 'Ozarks_and_Plains', 'Piedmont', 'Plains_and_Timbers', 'South_Texas_Plains', 'Texas_Blackland_Prairies', 'West_Gulf_Coastal_Plain', 'Central_Atlantic_Coastal_Plain', 'Central_Gulf_Coastal_Plain', 'Chihuahuan_Deserts0', 'East_Gulf_Coastal_Plain_Sub', 'Edwards_Plateau0', 'Florida_Peninsula0', 'Gulf_Coastal_Prairies0', 'Interior_Plateau0', 'Marine_Gulf_Stream0', 'Marine_Shelf_and_Extension0', 'Mid_East_Gulf_Coastal_Plain', 'Mississippi_Alluvial_Valley0', 'North_Appalachians', 'North_Atlantic_Coastal_Plain', 'North_High_Plains_and_Tablelands', 'North_Missouri0', 'North_Piedmont', 'Ouachita0', 'Ozarks_and_Plains0', 'Plains_and_Timbers0', 'South_Appalachians', 'South_Atlantic_C

In [13]:
# too many feature classes! I have double copies where the LevelI subregions are not split into Level II subregions.
# here I'm making a new list of the ones I don't want, I'm doing this because I couldn't figure out how to do a negative wildcard
# what I really want is a lit of all subregions that don't end in 0
# above I checked to see if I could just overwrite instead of making the ones that end in 0, and it said it would overwrite
# but alas, it did not
FCListNeg =[]

for fc in FCList:
    if fnmatch.fnmatch(fc, "*0"):
        FCListNeg.append(fc)

In [14]:
print(FCListNeg)

['Chihuahuan_Deserts0', 'Edwards_Plateau0', 'Florida_Peninsula0', 'Gulf_Coastal_Prairies0', 'Interior_Plateau0', 'Marine_Gulf_Stream0', 'Marine_Shelf_and_Extension0', 'Mississippi_Alluvial_Valley0', 'North_Missouri0', 'Ouachita0', 'Ozarks_and_Plains0', 'Plains_and_Timbers0', 'South_Texas_Plains0', 'Texas_Blackland_Prairies0', 'West_Gulf_Coastal_Plain0']


In [15]:
# remove the ones I don't want from the list
FCListG = [x for x in FCList if x not in FCListNeg]

In [16]:
print(FCListG)

['Appalachians', 'Atlantic_Coastal_Plain', 'Chihuahuan_Deserts', 'East_Gulf_Coastal_Plain', 'Edwards_Plateau', 'Florida_Peninsula', 'Gulf_Coastal_Prairies', 'High_Plains_and_Tablelands', 'Interior_Plateau', 'Marine_Gulf_Stream', 'Marine_Shelf_and_Extension', 'Mississippi_Alluvial_Valley', 'North_Missouri', 'Ouachita', 'Ozarks_and_Plains', 'Piedmont', 'Plains_and_Timbers', 'South_Texas_Plains', 'Texas_Blackland_Prairies', 'West_Gulf_Coastal_Plain', 'Central_Atlantic_Coastal_Plain', 'Central_Gulf_Coastal_Plain', 'East_Gulf_Coastal_Plain_Sub', 'Mid_East_Gulf_Coastal_Plain', 'North_Appalachians', 'North_Atlantic_Coastal_Plain', 'North_High_Plains_and_Tablelands', 'North_Piedmont', 'South_Appalachians', 'South_Atlantic_Coastal_Plain', 'South_High_Plains_and_Tablelands', 'South_Piedmont']


In [17]:
# Loop convert subregions to polygons, snapping cells
for fc in FCListG:

     output = fc + "_R"

     # Process: Polygon to Raster
     with arcpy.EnvManager(snapRaster=SEraster):
        arcpy.PolygonToRaster_conversion(fc, "SubRgn_I", output, "CELL_CENTER", "", SEraster)

print ("finished polygon to raster") 

finished polygon to raster


In [18]:
# make a list of all subregions rasters
RList = arcpy.ListRasters() 

In [19]:
print(RList)

['Appalachians_R', 'Atlantic_Coastal_Plain_R', 'Chihuahuan_Deserts_R', 'East_Gulf_Coastal_Plain_R', 'Edwards_Plateau_R', 'Florida_Peninsula_R', 'Gulf_Coastal_Prairies_R', 'High_Plains_and_Tablelands_R', 'Interior_Plateau_R', 'Marine_Gulf_Stream_R', 'Marine_Shelf_and_Extension_R', 'Mississippi_Alluvial_Valley_R', 'North_Missouri_R', 'Ouachita_R', 'Ozarks_and_Plains_R', 'Piedmont_R', 'Plains_and_Timbers_R', 'South_Texas_Plains_R', 'Texas_Blackland_Prairies_R', 'West_Gulf_Coastal_Plain_R', 'Central_Atlantic_Coastal_Plain_R', 'Central_Gulf_Coastal_Plain_R', 'East_Gulf_Coastal_Plain_Sub_R', 'Mid_East_Gulf_Coastal_Plain_R', 'North_Appalachians_R', 'North_Atlantic_Coastal_Plain_R', 'North_High_Plains_and_Tablelands_R', 'North_Piedmont_R', 'South_Appalachians_R', 'South_Atlantic_Coastal_Plain_R', 'South_High_Plains_and_Tablelands_R', 'South_Piedmont_R']


### Roads

In [20]:
# select out the larger roads from the file created in R from the openstreetmap download
# these are larger roads that we definately want to capture some aspect of
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Select(roads, "roadsLarge", "fclass NOT IN ('secondary','secondary_link','tertiary','tertiary_link','residential')")

In [21]:
# add and calculate a field that we can use to convert to a raster
arcpy.management.CalculateField("roadsLarge", "dissolve", "1", "PYTHON3", '', "SHORT")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_SubregionsAndEdges\\SubregionsAndEdgesForWorkshops.gdb\\roadsLarge'>

In [22]:
# convert lines to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("roadsLarge", "dissolve", "roadsLargeR", SEraster)

In [23]:
# select out the smaller roads from the file created in R from the openstreetmap download
# these are smaller roads, but ones that sometimes get picked up in the NLCD low intensity urban class
# I'm pulling them out incase they are important to create edges in rural areas
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Select(roads, "roadsMed", "fclass IN ('secondary','secondary_link','tertiary','tertiary_link')")

In [24]:
# add and calculate a field that we can use to convert to a raster
arcpy.management.CalculateField("roadsMed", "dissolve", "1", "PYTHON3", '', "SHORT")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_SubregionsAndEdges\\SubregionsAndEdgesForWorkshops.gdb\\roadsMed'>

In [25]:
# convert lines to raster
# I'm coverting these smaller roads to a 10 meter raster, then resampling, so they don't form full linear features
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.conversion.FeatureToRaster("roadsMed", "dissolve", "roadsMedR", 10)

In [26]:
# resample up to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.Resample("roadsMedR", "roadsMedR30", "30 30", "NEAREST")

In [27]:
# combine the roads layers
out_raster = arcpy.sa.CellStatistics("roadsLargeR;roadsMedR30", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("roads2")

In [28]:
# Make a raster with the 3 road as 0 and everything else is 1
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Reclassify("roads2", "Value", "1 0;NODATA 1", "DATA"); out_raster.save("roads2z")

### NLCD urban

In [29]:
# Make a raster with the 2 highest urban classes as 0 and everything else is 1
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con(nlcd, 0, 1, "Value = 23 Or Value = 24"); out_raster.save("urb")

### reservoirs

In [30]:
# make a raster with reservoris as 0 and everything else as 1 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Reclassify(resNull, "VALUE", "1 1;0 0;NODATA 0", "DATA"); out_raster.save("res2")

In [31]:
# this should make a raster with both reservoirs and urban as 0 and everything else as 1
#out_raster = arcpy.sa.Times("res", "urb"); out_raster.save("UrbRes")

In [32]:
# this should make a raster with roads, reservoirs, or urban as 0 and everything else as 1
out_raster = arcpy.sa.CellStatistics("res2;urb;roads2z", "MINIMUM", "DATA", "SINGLE_BAND"); out_raster.save("UrbResRoad")

In [33]:
# make a raster with roads, urban or reservoris as NODATA and everything else as 1 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Reclassify("UrbResRoad", "VALUE", "1 1;0 NODATA", "DATA"); out_raster.save("UrbResRoadND")

### make subregion masks for use in zonation
These masks have NoData for urban and reservoirs and 1 for the areas inside the subregion we want zonation to prioritize

In [34]:
# Loop through all rasters, use times to remove urban and reservoirs from subregions
for r in RList:
   
    # Process: 
    with arcpy.EnvManager(snapRaster=SEraster):
        out_raster = arcpy.sa.Times(r, "UrbResRoadND"); out_raster.save(OutWorkspace2 + "\\" + r + "Z.tif")

print ("finished making zonation masks") 

finished making zonation masks


### make subregion masks to rebalance zonation results
These masks have 0 for urban and reservoirs, 1 for the areas inside the subregion we want zonation to prioritize, NoData outside the subregion

In [35]:
# Loop through all rasters, use times to remove urban and reservoirs from subregions
for r in RList:
   
    # Process: 
    with arcpy.EnvManager(snapRaster=SEraster):
        out_raster = arcpy.sa.Times(r, "UrbResRoad"); out_raster.save(OutWorkspace2 + "\\" + r + ".tif")

print ("finished making zonation rebalancing rasters") 

finished making zonation rebalancing rasters


### Export roads layer as .tif

In [36]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("roads2", OutWorkspace3 + "\\" + "roads.tif", '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [37]:
# clean up space, delete files
#for fc in FCList:
#    arcpy.Delete_management(fc)
    
#print ("finished deleting")

In [38]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

7986.33199596405
